In [2]:
import logging
import pandas as pd
import os
import json
import time
import requests

In [3]:
with open("session.txt", "r") as f:
    sessionid = f.read()

In [17]:
headers_for_html_fetching = {
            'Authority': 'www.threads.net',
            'Accept': (
                'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;'
                'q=0.8,application/signed-exchange;v=b3;q=0.7'
            ),
            'Accept-Language': 'en-US,en;q=0.9',
            'Cache-Control': 'no-cache',
            'Content-Type': 'application/x-www-form-urlencoded',
            'Origin': 'https://www.threads.net',
            'Pragma': 'no-cache',
            'Referer': 'https://www.instagram.com',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'cross-site',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': (
                'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) '
                'AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1.2 Safari/605.1.15'
            ),
        }

In [18]:
import re
def get_threads_userpk(username:str):
    cookies = {
            "sessionid": sessionid
    }
    response = requests.get(
                url=f'https://www.threads.net/@{username}',
                headers=headers_for_html_fetching,
                cookies=cookies
            )
    try:
        if "viewer does not follow private user" in response.text:
            return None
        text = response.text
        user_id_key_value = re.search('"user_id":"(\\d+)",', text).group()
        user_id = re.search('\\d+', user_id_key_value).group()
    except:
        raise Exception()
    return user_id

get_threads_userpk("shelli1370")

'63074975810'

In [1]:
from retrieve_data_by_query import RetrieveDataByQuery
t = RetrieveDataByQuery(sessionid=sessionid)
# t.retrieve_follower_by_userid_after("69655371289","0")

NameError: name 'sessionid' is not defined

In [20]:

def get_follower_data(data: list):
    try:
        return [
            {
                "pk": node.get("pk", ""),
                "username": node.get("username", ""),
                "follower_count": node.get("follower_count", "")
            }
            for d in data
            if (node := d.get("node", {}))
        ]
    except Exception as e:
        raise RuntimeError(f"Errore durante l'elaborazione dei dati dei follower: {e}")


progress_tracker = {}
filejson = "progress2.json"

def save_progress(userid: str, batch_number: int, results: list):
    """Salva il batch_number corrente e i risultati ottenuti per un determinato userid."""
    progress_tracker[userid] = {"batch_number": batch_number, "results": results}
    with open(filejson, "w") as f:
        json.dump(progress_tracker, f)

def load_progress(userid: str):
    """Carica il batch_number e i risultati salvati per un determinato userid."""
    try:
        with open(filejson, "r") as f:
            data = json.load(f)
            return data.get(userid, {"batch_number": 1, "results": []})
    except (FileNotFoundError, json.JSONDecodeError):
        return {"batch_number": 1, "results": []}
def remove_progress(userid: str):
    """Rimuove il progresso salvato per un determinato userid."""
    try:
        with open(filejson, "r") as f:
            data = json.load(f)
        
        if userid in data:
            del data[userid]
            with open(filejson, "w") as f:
                json.dump(data, f)
    except (FileNotFoundError, json.JSONDecodeError):
        pass
    
def get_all_follower(userid: str):
    progress = load_progress(userid)
    batch_number = progress["batch_number"]
    results = progress["results"]
    allpk = [res["pk"] for res in results]
    try:
        has_next_page = True
        while has_next_page:
            try:
                print("Pagina:", batch_number)
                response = t.retrieve_follower_by_userid_after(userid, str(25 * batch_number-1))
                batch_number += 1
                data = response.get("data", {}).get("fetch__XDTUserDict", {}).get("followers", {})
                if data.get("edges", []) == []:
                    return [{
                        "pk": userid,
                        "username": None,
                        "follower_count": None
                    }]
                d_out = get_follower_data(data.get("edges", []))
                for res in d_out:
                    if res["pk"] in allpk:
                        remove_progress(userid)
                        return results
                    allpk.append(res["pk"])
                results.extend(d_out)
                
                has_next_page = data.get("page_info", {}).get("has_next_page", False)
                
                # Salva il progresso dopo ogni batch
                save_progress(userid, batch_number, results)
                
            except Exception as e:
                save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore
                raise RuntimeError(f"Errore nel recupero della paginazione dei follower: {e}")
    
    except Exception as e:
        save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore generale
        raise RuntimeError(f"Errore generale nel recupero dei follower per l'utente {userid}: {e}")
    remove_progress(userid)
    return results

In [21]:
df = pd.read_csv("fol1.csv")
df['follower_pk'] = df['follower_pk'].astype(str)
unique_user_pk = df['follower_pk'].unique()
len(unique_user_pk)

1628

In [22]:
dff = pd.read_csv("followers_MachineLearning.csv")
dff.head(1)

,user_pk,follower_pk,follower_username,follower_count
0,64525247485,4529033111,mbahareth,184965


In [23]:
def load_processed_users(file_path):
    if os.path.exists(file_path):
        logging.info(f"Loading processed users from {file_path}")
        df = pd.read_csv(file_path)
        df['user_pk'] = df['user_pk'].astype(str)
        return set(df['user_pk'].tolist())
    return set()

def save_to_csv(df, file_path):
    try:
        if os.path.exists(file_path):
            df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(file_path, index=False)
        logging.info(f"Saved {len(df)} records to {file_path}")
    except Exception as e:
        logging.error(f"Error saving data to {file_path}: {e}")
        
csv_file = "ffML1.csv"
processed_users = load_processed_users(csv_file)

In [24]:
len(processed_users)

0

In [25]:
print(len(unique_user_pk))
unique_user_pk = unique_user_pk[unique_user_pk != "339473819"]

print(len(unique_user_pk))

1628
1628


In [ ]:
def fetch_followers(pk,user_pk):
    print(f"Fetching followers for user_pk: {pk}")
    try:
        follower_data = get_all_follower(pk)
        return [
            {
                "user_pk": user_pk,
                "follower_pk": follower["pk"],
                "follower_username": follower["username"],
                "follower_count": follower["follower_count"],
                "user_threads_userpk": pk
            }
            for follower in follower_data
        ]
    except Exception as e:
        logging.error(f"Error fetching followers for {pk}: {e}")
        return None

follower_list = []
batch_size = 1


print("Starting follower data collection")
for user_pk in unique_user_pk:
    if user_pk in processed_users:
        print(f"Skipping already processed user_pk: {user_pk}")
        continue
    processed_users.update([user_pk])
    username= dff[dff["follower_pk"] == int(user_pk)]["follower_username"].unique()[0]
    print(user_pk, username)
    threads_user_pk = get_threads_userpk(str(username))
    if threads_user_pk is None:
        follower_list.extend([{
                "user_pk": user_pk,
                "follower_pk": None,
                "follower_username": None,
                "follower_count": None,
                "user_threads_userpk": None
        }])
        continue
    results = fetch_followers(threads_user_pk,user_pk)
    if results is None:
        break
    follower_list.extend(results)
    
    if follower_list:
        print(f"Processing batch of {len(follower_list)} followers")
        follower_df = pd.DataFrame(follower_list)
        save_to_csv(follower_df, csv_file)
        follower_list = []

if follower_list:
    print(f"Saving remaining {len(follower_list)} followers")
    follower_df = pd.DataFrame(follower_list)   
    save_to_csv(follower_df, csv_file)

print("Follower data collection completed")


In [27]:
df1 = pd.read_csv(csv_file)
df_count= df1.drop_duplicates().groupby("user_pk").count()
print(df_count.shape)
df_count

(1, 4)


,follower_pk,follower_username,follower_count,user_threads_userpk
user_pk,,,,
4529033111,49,49,49,49


In [6]:
import pandas as pd

In [22]:
pfd = pd.read_csv("post_data_machine_learning.csv")
pfd

,id,post_pk,caption_text,like_count,taken_at,username,user_pk,quote_count,repost_count,reshare_count
0,3558989852217342995_4529033111,3.558990e+18,نموذج تحقيق الأهداف \nGoal Model Canvas ©️,2.0,1.738485e+09,mbahareth,4529033111,0.0,0.0,NaN
1,3558979884327928742_4529033111,3.558980e+18,“البنية التحتية هي الأساس لأي تطور تقني. 🚀 إذا...,1.0,1.738483e+09,mbahareth,4529033111,0.0,0.0,NaN
2,3558972290465133071_4529033111,3.558972e+18,احصل على عروض حصرية لتأجير السيارات في إيجارو ...,0.0,1.738483e+09,mbahareth,4529033111,0.0,0.0,NaN
3,3558612126880967013_4529033111,3.558612e+18,المشاهير يجذبون الجماهير بمحتواهم، لكنني صنعت ...,1.0,1.738440e+09,mbahareth,4529033111,0.0,0.0,NaN
4,3558524801563818044_4529033111,3.558525e+18,معد تحتاج ميكرفون في iOS 18.3,1.0,1.738429e+09,mbahareth,4529033111,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
70840,3331793971820207339_47667908711,3.331794e+18,Essi sono stati capaci di sacrificarsi affinch...,6.0,1.711401e+09,iulusianualis,47667908711,0.0,0.0,NaN
70841,3330794266533349418_47667908711,3.330794e+18,https://www.commentimemorabili.it/social-roule...,0.0,1.711282e+09,iulusianualis,47667908711,0.0,0.0,NaN
70842,3330794298879807638_47667908711,3.330794e+18,Io mi ritengo femminista e sono favorevole all...,3.0,1.711282e+09,iulusianualis,47667908711,0.0,0.0,NaN
70843,3329563727440876588_47667908711,3.329564e+18,Cerco polemica\nCredete che libertà ed uguagli...,1.0,1.711135e+09,iulusianualis,47667908711,0.0,0.0,NaN


In [70]:
followerai

,thread_user_pk,thread_follower_pk,thread_username,thread_follower_username
0,6.331078e+10,6.408954e+10,mlssfshn,gabriel_lagon
1,6.338794e+10,6.965537e+10,thisisaltgian,beiyo_
2,6.338794e+10,6.313327e+10,thisisaltgian,realwanita
3,6.338794e+10,6.322924e+10,thisisaltgian,pelumi__i
4,6.338794e+10,6.342134e+10,thisisaltgian,olufisola_
...,...,...,...,...
25182,6.329816e+10,6.322772e+10,msmarshellw,corinnespartana
25183,6.798282e+10,6.322772e+10,affyayomikun,corinnespartana
25184,6.825805e+10,6.322772e+10,quentin.d.h,corinnespartana
25185,6.325159e+10,6.322772e+10,ohnelleyeah,corinnespartana


In [65]:
followerai = pd.read_csv("followersAI.csv")
followerai
user = list(followerai["thread_username"]) + list(followerai["thread_follower_username"])
user

['mlssfshn',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian',
 'thisisaltgian

In [71]:
df = pd.read_csv("ffML_merged.csv")
f1 = df[df["follower_username"].isin(user) | df["user_username"].isin(user)]
f1
# df[["user_threads_userpk","user_username","follower_pk","follower_username"]].drop_duplicates()


,user_pk,follower_pk,follower_username,follower_count,user_threads_userpk,user_username
101,622028921,6.325577e+10,paulmoreira_,2295.0,6.325991e+10,yourmomsasaurasrekt
148,468639049,6.321661e+10,aitoolswhitehattoolbox,1298.0,6.329396e+10,gcanderoli
149,468639049,6.969695e+10,lilomisaki01,0.0,6.329396e+10,gcanderoli
150,468639049,6.307371e+10,tinchobenegas,395.0,6.329396e+10,gcanderoli
151,468639049,6.413115e+10,cok_ramos,100.0,6.329396e+10,gcanderoli
...,...,...,...,...,...,...
455350,61371375385,6.283742e+10,penelope1_4__zimmer,775.0,6.343985e+10,mymodelsinfo
455627,54516422271,6.534487e+10,coffeebeansdev,6600.0,6.355063e+10,alessio.ottaviani.art
455827,51612188881,6.358229e+10,vitosrh,4558.0,6.354236e+10,luca__ausilio
456913,34173187368,6.297742e+10,chiara.bass51,2513.0,6.370134e+10,matteo_volpone


In [73]:
f2 = df[~df["follower_username"].isin(user) | ~df["user_username"].isin(user)].head(5000)
f2

,user_pk,follower_pk,follower_username,follower_count,user_threads_userpk,user_username
0,4529033111,7.252719e+10,koroshshirdel621,7.0,6.345592e+10,mbahareth
1,4529033111,6.875476e+10,hanansaleh3564,74.0,6.345592e+10,mbahareth
2,4529033111,7.221591e+10,mohammad_ezz_,110.0,6.345592e+10,mbahareth
3,4529033111,7.171160e+10,rahma.abdelmageed.98,40.0,6.345592e+10,mbahareth
4,4529033111,6.346844e+10,bouzguenda_training_center,1398.0,6.345592e+10,mbahareth
...,...,...,...,...,...,...
4995,3566457318,6.709093e+10,runntrailgr,720.0,6.427558e+10,andreas_vrl
4996,3566457318,6.914883e+10,franky_woodward,268.0,6.427558e+10,andreas_vrl
4997,3566457318,6.821379e+10,irinazhardan,1455.0,6.427558e+10,andreas_vrl
4998,3566457318,6.561051e+10,venetia_gkonoy,831.0,6.427558e+10,andreas_vrl


In [98]:
f3 = pd.concat([f1,f2])
f3 = f3[["user_threads_userpk","user_username","follower_pk","follower_username"]]
f3.rename(
    {
        "user_threads_userpk":"thread_user_pk",
        "user_username":"thread_username",
        "follower_pk":"thread_follower_pk",
        "follower_username": "thread_follower_username"
    }
    ,inplace = True,
    axis = 1
)
f3

,thread_user_pk,thread_username,thread_follower_pk,thread_follower_username
101,6.325991e+10,yourmomsasaurasrekt,6.325577e+10,paulmoreira_
148,6.329396e+10,gcanderoli,6.321661e+10,aitoolswhitehattoolbox
149,6.329396e+10,gcanderoli,6.969695e+10,lilomisaki01
150,6.329396e+10,gcanderoli,6.307371e+10,tinchobenegas
151,6.329396e+10,gcanderoli,6.413115e+10,cok_ramos
...,...,...,...,...
4995,6.427558e+10,andreas_vrl,6.709093e+10,runntrailgr
4996,6.427558e+10,andreas_vrl,6.914883e+10,franky_woodward
4997,6.427558e+10,andreas_vrl,6.821379e+10,irinazhardan
4998,6.427558e+10,andreas_vrl,6.561051e+10,venetia_gkonoy


In [97]:
df2 = pd.read_csv("followers_MachineLearning.csv")
df2 = df2[["user_threads_userpk","username","user_threads_follower_pk","follower_username"]]
df2.rename(
    {
        "user_threads_userpk":"thread_user_pk",
        "username":"thread_username",
        "user_threads_follower_pk":"thread_follower_pk",
        "follower_username": "thread_follower_username"
    }
    ,inplace = True,
    axis = 1
)
df2

,thread_user_pk,thread_username,thread_follower_pk,thread_follower_username
0,6.345592e+10,NaN,6.345592e+10,mbahareth
1,6.361918e+10,NaN,6.361918e+10,subflw47
2,6.536388e+10,NaN,6.536388e+10,pasperfection
3,6.325991e+10,NaN,6.325991e+10,yourmomsasaurasrekt
4,6.387085e+10,NaN,6.387085e+10,bezprzesady.eth
...,...,...,...,...
3760,6.355556e+10,NaN,6.355556e+10,rebeccamatoni
3761,6.386989e+10,NaN,6.386989e+10,mikymolino
3762,6.386901e+10,NaN,6.386901e+10,hysni_9
3763,6.299286e+10,NaN,6.299286e+10,mario.__.dauria


In [100]:
dfinal = pd.concat([df2, f3])
dfinal.to_csv("followersML.csv")